In [1]:
room_config = {
    "officeroom": {
        "north": "wall",
        "east": "livingroom",
        "south": "wall",
        "west": "wall",
    },
    "livingroom": {
        "north": "wall",
        "east": "wall",
        "south": "bedroom",
        "west": "officeroom",
    },
    "bedroom": {
        "north": "livingroom",
        "east": "wall",
        "south": "wall",
        "west": "wall",
    },
}


object_transition_config = {
    "static": {"bed": None, "desk": None, "table": None},
    "independent": {
        "tae": {
            "officeroom": {"north": 0, "east": 0.1, "south": 0, "west": 0, "stay": 0.9},
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0,
                "west": 0.1,
                "stay": 0.9,
            },
            "bedroom": {"north": 0, "east": 0, "south": 0, "west": 0, "stay": 0},
        },
        "michael": {
            "officeroom": {
                "north": 0,
                "east": 0,
                "south": 0,
                "west": 0,
                "stay": 0,
            },
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0.9,
                "west": 0,
                "stay": 0.1,
            },
            "bedroom": {"north": 0.1, "east": 0, "south": 0, "west": 0, "stay": 0.9},
        },
        "vincent": {
            "officeroom": {
                "north": 0,
                "east": 0.5,
                "south": 0,
                "west": 0,
                "stay": 0.5,
            },
            "livingroom": {
                "north": 0,
                "east": 0,
                "south": 0.333,
                "west": 0.333,
                "stay": 0.333,
            },
            "bedroom": {
                "north": 0.5,
                "east": 0,
                "south": 0,
                "west": 0,
                "stay": 0.5,
            },
        },
    },
    "dependent": {
        "laptop": {"tae": 0.7, "michael": 0.4, "vincent": 0.1},
        "phone": {"tae": 0.1, "michael": 0.7, "vincent": 0.4},
        "headset": {"tae": 0.4, "michael": 0.1, "vincent": 0.9},
    },
    "agent": {
        "agent": {"officeroom": None, "livingroom": None, "bedroom": None},
    },
}

object_init_config = {
    "static": {
        "bed": {"officeroom": 0, "livingroom": 0, "bedroom": 1},
        "desk": {"officeroom": 1, "livingroom": 0, "bedroom": 0},
        "table": {"officeroom": 0, "livingroom": 1, "bedroom": 0},
    },
    "independent": {
        "tae": {"officeroom": 0.5, "livingroom": 0.5, "bedroom": 0},
        "michael": {"officeroom": 0, "livingroom": 0.5, "bedroom": 0.5},
        "vincent": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
    "dependent": {
        "laptop": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
        "phone": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
        "headset": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
    "agent": {
        "agent": {"officeroom": 0.333, "livingroom": 0.333, "bedroom": 0.333},
    },
}

config = {
    "room_config": room_config,
    "object_transition_config": object_transition_config,
    "object_init_config": object_init_config,
    "question_prob": 1.0,
    "seed": 42,
    "terminates_at": 99,
}

In [ ]:
import gymnasium as gym
import random

env = gym.make("room_env:RoomEnv-v2", **config)
(obs, question), info = env.reset()
while True:
    action_qa = "livingroom"
    action_explore = random.choice(["north", "east", "south", "west", "stay"])
    (obs, question), reward, done, truncated, info = env.step(("wall", action_explore))
    if done:
        break

In [3]:
obs, question

([['livingroom', 'tothenorth', 'wall', 100],
  ['livingroom', 'totheeast', 'wall', 100],
  ['livingroom', 'tothesouth', 'bedroom', 100],
  ['livingroom', 'tothewest', 'officeroom', 100],
  ['table', 'atlocation', 'livingroom', 100],
  ['vincent', 'atlocation', 'livingroom', 100],
  ['headset', 'atlocation', 'livingroom', 100],
  ['agent', 'atlocation', 'livingroom', 100]],
 ['table', 'atlocation', '?', 100])

In [4]:
env.hidden_global_state

[['officeroom', 'tothenorth', 'wall', 100],
 ['officeroom', 'totheeast', 'livingroom', 100],
 ['officeroom', 'tothesouth', 'wall', 100],
 ['officeroom', 'tothewest', 'wall', 100],
 ['livingroom', 'tothenorth', 'wall', 100],
 ['livingroom', 'totheeast', 'wall', 100],
 ['livingroom', 'tothesouth', 'bedroom', 100],
 ['livingroom', 'tothewest', 'officeroom', 100],
 ['bedroom', 'tothenorth', 'livingroom', 100],
 ['bedroom', 'totheeast', 'wall', 100],
 ['bedroom', 'tothesouth', 'wall', 100],
 ['bedroom', 'tothewest', 'wall', 100],
 ['bed', 'atlocation', 'bedroom', 100],
 ['desk', 'atlocation', 'officeroom', 100],
 ['table', 'atlocation', 'livingroom', 100],
 ['tae', 'atlocation', 'officeroom', 100],
 ['michael', 'atlocation', 'bedroom', 100],
 ['vincent', 'atlocation', 'livingroom', 100],
 ['laptop', 'atlocation', 'officeroom', 100],
 ['phone', 'atlocation', 'bedroom', 100],
 ['headset', 'atlocation', 'livingroom', 100],
 ['agent', 'atlocation', 'livingroom', 100]]

In [5]:
import datetime
import random
from explicit_memory.memory import EpisodicMemory, SemanticMemory, ShortMemory


class HandCraftedAgent:
    """Handcrafted agent interacting with environment.

    This agent explores the roooms, i.e., KGs. The exploration can be uniform-random,
    or just avoiding walls.

    """

    def __init__(
        self,
        env_str: str = "room_env:RoomEnv-v2",
        env_config: dict = None,
        policy: str = "random",
        num_samples_for_results: int = 10,
        test_seed: int = 42,
        capacity: dict = {
            "episodic": 8,
            "semantic": 8,
            "short": 32,
        },
    ) -> None:
        """Initialize the agent.

        Args
        ----
        env_str: This has to be "room_env:RoomEnv-v2"
        env_config: The configuration of the environment.
        policy: The room exploration policy. Choose one of "random", "avoid_walls"
        num_samples_for_results: The number of samples to validate / test the agent.
        test_seed: The random seed for test.
        capacity: The capacity of each human-like memory systems.

        """
        self.all_params = deepcopy(locals())
        del self.all_params["self"]
        self.env_str = env_str
        self.env_config = env_config
        self.env_config["seed"] = test_seed
        self.policy = policy
        assert self.policy in ["random", "avoid_walls"]
        self.num_samples_for_results = num_samples_for_results
        self.test_seed = test_seed
        self.capacity = capacity

        self.env = gym.make(self.env_str, **env_config)

        self.default_root_dir = f"./training_results/{str(datetime.datetime.now())}"
        os.makedirs(self.default_root_dir, exist_ok=True)

    def init_memory_systems(self, num_actions: int = 5) -> None:
        """Initialize the agent's memory systems. This has nothing to do with the
        replay buffer."""
        self.action_space = gym.spaces.Discrete(num_actions)
        self.memory_systems = {
            "episodic": EpisodicMemory(capacity=self.capacity["episodic"]),
            "semantic": SemanticMemory(capacity=self.capacity["semantic"]),
            "short": ShortMemory(capacity=self.capacity["short"]),
        }

In [6]:
memory_systems = {
    "episodic": EpisodicMemory(capacity=4),
    "semantic": SemanticMemory(capacity=4),
    "short": ShortMemory(capacity=2),
}

In [7]:
import gymnasium as gym
import random

env = gym.make("room_env:RoomEnv-v2", **config)
(obs, question), info = env.reset()
obs_len = []
obs_len.append(len(obs))
obs_all = []
obs_all.append(obs)
while True:
    action_qa = "livingroom"
    action_explore = random.choice(["north", "east", "south", "west", "stay"])
    (obs, question), reward, done, truncated, info = env.step(("wall", action_explore))
    if done:
        break
    obs_len.append(len(obs))
    obs_all.append(obs)

In [8]:
foo = EpisodicMemory(4, False)
foo.add(
    {"human": None, "object": "tae", "object_location": "livingroom", "timestamp": 0}
)
foo

2023-09-05 18:12:55.372 INFO memory - add: memory entry {'human': None, 'object': 'tae', 'object_location': 'livingroom', 'timestamp': 0} added. Now there are in total of 1 memories!


{   '_frozen': False,
    'capacity': 4,
    'entries': [   {   'human': None,
                       'object': 'tae',
                       'object_location': 'livingroom',
                       'timestamp': 0}],
    'remove_duplicates': False,
    'type': 'episodic'}

In [9]:
foo.add(
    {"human": None, "object": "tae", "object_location": "officeroom", "timestamp": 0}
)

foo

2023-09-05 18:12:55.381 INFO memory - add: memory entry {'human': None, 'object': 'tae', 'object_location': 'officeroom', 'timestamp': 0} added. Now there are in total of 2 memories!


{   '_frozen': False,
    'capacity': 4,
    'entries': [   {   'human': None,
                       'object': 'tae',
                       'object_location': 'livingroom',
                       'timestamp': 0},
                   {   'human': None,
                       'object': 'tae',
                       'object_location': 'officeroom',
                       'timestamp': 0}],
    'remove_duplicates': False,
    'type': 'episodic'}

In [4]:
sorted([[1, 2, 3], ["foo", "bar", -1]], key=lambda x: x[-1], reverse=True)

[[1, 2, 3], ['foo', 'bar', -1]]

In [3]:
foo = [1, 2, 3, 0]

bar = sorted(foo)

foo, bar

([1, 2, 3, 0], [0, 1, 2, 3])

In [10]:
foo.add(
    {"human": None, "object": "tae", "object_location": "livingroom", "timestamp": 1}
)

foo

2023-09-05 18:12:55.388 INFO memory - add: memory entry {'human': None, 'object': 'tae', 'object_location': 'livingroom', 'timestamp': 1} added. Now there are in total of 3 memories!


{   '_frozen': False,
    'capacity': 4,
    'entries': [   {   'human': None,
                       'object': 'tae',
                       'object_location': 'livingroom',
                       'timestamp': 0},
                   {   'human': None,
                       'object': 'tae',
                       'object_location': 'officeroom',
                       'timestamp': 0},
                   {   'human': None,
                       'object': 'tae',
                       'object_location': 'livingroom',
                       'timestamp': 1}],
    'remove_duplicates': False,
    'type': 'episodic'}

In [11]:
mems = [
    ["foo", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
]

In [12]:
mems.remove(["foo", "atlocation", "bar", 0])

In [13]:
mems

[['foo', 'atlocation', 'bad', 1]]

In [14]:
foo.return_as_lists()

[{'human': None,
  'object': 'tae',
  'object_location': 'livingroom',
  'timestamp': 0},
 {'human': None,
  'object': 'tae',
  'object_location': 'officeroom',
  'timestamp': 0},
 {'human': None,
  'object': 'tae',
  'object_location': 'livingroom',
  'timestamp': 1}]

In [7]:
["bob", "atlocation", "?", 42].index("?")

2

In [3]:
search = ["?", "atlocation", "bar", 42]
targets = [
    ["bob", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
    ["bob", "atlocation", "bar", 32],
    ["atlocation", "bar", "bar", 5],
]
candidates = []

for target in targets:
    assert len(search) == len(target) == 4
    count = 0

    for s, t in zip(search[:-1], target[:-1]):
        if s == t:
            count += 1

    if count == 2:
        candidates.append(target)

if len(candidates) > 0:
    print(candidates)
    candidates.sort(key=lambda x: x[-1])
    candidate = candidates[-1]
    candidate

[['bob', 'atlocation', 'bar', 0], ['bob', 'atlocation', 'bar', 32]]


In [13]:
search = ["bob", "atlocation", "bar", 12]

search, candidates

[
    candidate
    for candidate in candidates
    if "".join(search[:-1]) == "".join(candidate[:-1])
]

[['bob', 'atlocation', 'bar', 0], ['bob', 'atlocation', 'bar', 32]]

In [16]:
search[:-1] == candidates[0][:-1]

True

In [2]:
search = ["bob", "?", "bar", 42]
targets = [
    ["bob", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
    ["bob", "atlocation", "bar", 32],
    ["atlocation", "bob", "bar", 10],
    ["bar", "bob", "atlocation", 10],
    ["bob", "foo", "bar", 32],
]
candidates = [
    target for target in targets if len(set(search[:-1]).intersection(target[:-1])) == 2
]
candidates

[['bob', 'atlocation', 'bar', 0],
 ['bob', 'atlocation', 'bar', 32],
 ['atlocation', 'bob', 'bar', 10],
 ['bar', 'bob', 'atlocation', 10],
 ['bob', 'foo', 'bar', 32]]

In [41]:
set(search).intersection(targets[])


{'atlocation'}

In [1]:
foo = EpisodicMemory(4, False)

NameError: name 'EpisodicMemory' is not defined

In [25]:
targets = [
    ["bob", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
    ["bob", "atlocation", "bar", 32],
    ["atlocation", "bar", "bar", 5],
]

targets_ = [target[:-1] for target in targets]

targets_[0][0] = "tae"

targets_, targets

([['tae', 'atlocation', 'bar'],
  ['foo', 'atlocation', 'bad'],
  ['bob', 'atlocation', 'bar'],
  ['atlocation', 'bar', 'bar']],
 [['bob', 'atlocation', 'bar', 0],
  ['foo', 'atlocation', 'bad', 1],
  ['bob', 'atlocation', 'bar', 32],
  ['atlocation', 'bar', 'bar', 5]])

In [1]:
from explicit_memory.utils import list_duplicates_of

In [2]:
targets = [
    ["bob", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
    ["bob", "atlocation", "bar", 32],
    ["atlocation", "bar", "bar", 5],
    ["atlocation", "bar", "bar", 3],
    ["atlocation", "bar", "bar", 2],
]

targets_ = ["".join(target[:-1]) for target in targets]

to_remove = []
for target in set(targets_):
    indices = list_duplicates_of(targets_, target)
    if len(indices) > 1:
        print(indices)
        # to_remove.append(indices)

indices

[3, 4, 5]
[0, 2]


[0, 2]

In [24]:
targets = [
    ["bob", "atlocation", "bar", 33],
    ["bob", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
    ["bob", "atlocation", "bar", 32],
    ["atlocation", "bar", "bar", 1],
    ["atlocation", "bar", "bar", 6],
    ["atlocation", "bar", "bar", 2],
]

targets_ = ["".join(target[:-1]) for target in targets]

entries = ["".join(mem) for mem in targets_]  # to make list hashable
uniques = set(entries)

locs_all = [list_duplicates_of(entries, unique_entry) for unique_entry in uniques]
locs_all.sort(key=len)
entries_cleaned = []

for locs in locs_all:
    if len(locs) == 1:
        continue
    mem = targets[locs[0]]
    mem[-1] = max([targets[loc][-1] for loc in locs])
    entries_cleaned.append(mem)

entries_cleaned

[['atlocation', 'bar', 'bar', 6], ['bob', 'atlocation', 'bar', 33]]

In [25]:
targets.remove(targets[0])

targets

[['bob', 'atlocation', 'bar', 0],
 ['foo', 'atlocation', 'bad', 1],
 ['bob', 'atlocation', 'bar', 32],
 ['atlocation', 'bar', 'bar', 6],
 ['atlocation', 'bar', 'bar', 6],
 ['atlocation', 'bar', 'bar', 2]]

In [21]:
locs_all

[[2], [4, 5, 6], [0, 1, 3]]

In [22]:
from explicit_memory.memory import EpisodicMemory


em = EpisodicMemory(4, True)

for target in targets:
    em.add(target)

2023-09-05 22:39:39.956 INFO memory - add: memory entry ['bob', 'atlocation', 'bar', 33] added. Now there are in total of 1 memories!
2023-09-05 22:39:39.957 INFO memory - add: memory entry ['bob', 'atlocation', 'bar', 0] added. Now there are in total of 2 memories!
2023-09-05 22:39:39.958 INFO memory - add: memory entry ['foo', 'atlocation', 'bad', 1] added. Now there are in total of 2 memories!
2023-09-05 22:39:39.958 INFO memory - add: memory entry ['bob', 'atlocation', 'bar', 32] added. Now there are in total of 3 memories!
2023-09-05 22:39:39.958 INFO memory - add: memory entry ['atlocation', 'bar', 'bar', 6] added. Now there are in total of 3 memories!
2023-09-05 22:39:39.959 INFO memory - add: memory entry ['atlocation', 'bar', 'bar', 6] added. Now there are in total of 4 memories!
2023-09-05 22:39:39.960 INFO memory - add: memory entry ['atlocation', 'bar', 'bar', 2] added. Now there are in total of 4 memories!


In [23]:
em

{   '_frozen': False,
    'capacity': 4,
    'entries': [   [   'foo',
                       'atlocation',
                       'bad',
                       1],
                   [   'atlocation',
                       'bar',
                       'bar',
                       6],
                   [   'bob',
                       'atlocation',
                       'bar',
                       33]],
    'remove_duplicates': True,
    'type': 'episodic'}

In [29]:
"tae's laptop".split("'s")

['tae', ' laptop']

In [30]:
"laptop".split("'s")

['laptop']

In [33]:
import gymnasium as gym

env = gym.make("room_env:RoomEnv-v1")
env.des.semantic_knowledge

{'bowl': 'cupboard',
 'dog': 'kennel',
 'airplane': 'sky',
 'keyboard': 'desk',
 'elephant': 'circus',
 'sandwich': 'lunchbox',
 'boat': 'water',
 'handbag': 'store',
 'sheep': 'farm',
 'donut': 'bakery',
 'bicycle': 'garage',
 'bird': 'tree',
 'car': 'city',
 'oven': 'home',
 'kite': 'air',
 'train': 'zoo'}

In [35]:
[*[1, 2, 3], 5]

[1, 2, 3, 5]

In [5]:
from explicit_memory.memory import EpisodicMemory, SemanticMemory, ShortMemory


sm = SemanticMemory(4)

sm.add(["tae", "atlocation", "livingroom", 1])
sm.add(["tae", "atlocation", "livingroom", 2])
sm.add(["tae", "atlocation", "livingroom", 3])
sm.add(["tae", "atlocation", "livingroom", 4])
sm.entries.append(["tae", "atlocation", "livingroom", 5])
sm

2023-09-06 00:02:45.850 INFO memory - add: memory entry ['tae', 'atlocation', 'livingroom', 1] added. Now there are in total of 1 memories!
2023-09-06 00:02:45.851 INFO memory - add: memory entry ['tae', 'atlocation', 'livingroom', 2] added. Now there are in total of 2 memories!
2023-09-06 00:02:45.852 INFO memory - add: memory entry ['tae', 'atlocation', 'livingroom', 3] added. Now there are in total of 2 memories!
2023-09-06 00:02:45.853 INFO memory - add: memory entry ['tae', 'atlocation', 'livingroom', 4] added. Now there are in total of 2 memories!


{   '_frozen': False,
    'capacity': 4,
    'entries': [   [   'tae',
                       'atlocation',
                       'livingroom',
                       10],
                   [   'tae',
                       'atlocation',
                       'livingroom',
                       5]],
    'type': 'semantic'}

In [6]:
sm.clean_same_memories()
sm

{   '_frozen': False,
    'capacity': 4,
    'entries': [   [   'tae',
                       'atlocation',
                       'livingroom',
                       15]],
    'type': 'semantic'}

In [1]:
"Tae's food".split("'s")

['Tae', ' food']

In [5]:
name_entity = "Tae's food"

name, entity = name_entity.split("'s ")

In [6]:
name, entity

('Tae', 'food')

In [9]:
remove_posession("tae's food")

'food'

In [ ]:
from explicit_memory.utils import remove_posession, remove_timestamp

MARKER = "^^^"
entries = [
    ["bob's food", "atlocation", "bar", 33],
    ["tae's food", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
    ["bob", "atlocation", "bar", 32],
    ["atlocation", "bar", "kid's bar", 1],
    ["atlocation", "bar", "michael's bar", 6],
    ["atlocation", "bar", "vincent's bar", 2],
    ["atlocation", "bar", "bar", 2],
    ["atlocation", "bar", "bar", 2],
]

# -1 removes the timestamps from the quadruples
semantic_possibles = [
    [remove_posession(e) for e in remove_timestamp(entry)] for entry in entries
]
# MARKER is to allow hashing.
semantic_possibles = [MARKER.join(elem) for elem in semantic_possibles]


def duplicates(mylist, item):
    return [i for i, x in enumerate(mylist) if x == item]


semantic_possibles = dict(
    (x, duplicates(semantic_possibles, x)) for x in set(semantic_possibles)
)

if len(semantic_possibles) == len(entries):
    print("no episodic memories found to be compressible.")
    # return None, None
elif len(semantic_possibles) < len(entries):
    # logging.debug("some episodic memories found to be compressible.")

    max_key = max(semantic_possibles, key=lambda k: len(semantic_possibles[k]))
    indexes = semantic_possibles[max_key]

    episodic_memories = map(entries.__getitem__, indexes)
    episodic_memories = list(episodic_memories)
    # sort from the oldest to the latest
    episodic_memories = sorted(episodic_memories, key=lambda x: x[-1])
    semantic_memory = max_key.split(MARKER)
    # num_generalized_memories is the number of compressed episodic memories.
    semantic_memory.append(len(indexes))
    assert (len(semantic_memory)) == 4
    for mem in episodic_memories:
        assert len(mem) == 4

    print(
        f"{len(indexes)} episodic memories can be compressed "
        f"into one semantic memory: {semantic_memory}."
    )

    # return episodic_memories, semantic_memory
else:
    raise ValueError

In [33]:
import random

entries = [
    ["bob's food", "atlocation", "bar", 33],
    ["tae's food", "atlocation", "bar", 0],
    ["foo", "atlocation", "bad", 1],
    ["bob", "atlocation", "bar", 32],
    ["atlocation", "bar", "kid's bar", 1],
    ["atlocation", "bar", "michael's bar", 6],
    ["atlocation", "bar", "vincent's bar", 2],
    ["atlocation", "bar", "bar", 2],
    ["atlocation", "bar", "bar", 2],
]
best_semantic_possibles = []

for mem in entries:
    head, relation, tail = (
        remove_posession(mem[0]),
        remove_posession(mem[1]),
        remove_posession(mem[2]),
    )
    best_semantic_possibles.append([head, relation, tail])

best_semantic_possibles = [
    (i, elem, best_semantic_possibles.count(elem))
    for i, elem in enumerate(best_semantic_possibles)
]

In [34]:
best_semantic_possibles

[(0, ['food', 'atlocation', 'bar'], 2),
 (1, ['food', 'atlocation', 'bar'], 2),
 (2, ['foo', 'atlocation', 'bad'], 1),
 (3, ['bob', 'atlocation', 'bar'], 1),
 (4, ['atlocation', 'bar', 'bar'], 5),
 (5, ['atlocation', 'bar', 'bar'], 5),
 (6, ['atlocation', 'bar', 'bar'], 5),
 (7, ['atlocation', 'bar', 'bar'], 5),
 (8, ['atlocation', 'bar', 'bar'], 5)]

In [38]:
highest_freq = max([elem[2] for elem in best_semantic_possibles])

best_semantic_possibles = [
    elem for elem in best_semantic_possibles if elem[2] == highest_freq
]

mem_sem = random.choice(best_semantic_possibles)
idx = mem_sem[0]

mem_selected = entries[idx]

In [39]:
best_semantic_possibles

[(4, ['atlocation', 'bar', 'bar'], 5),
 (5, ['atlocation', 'bar', 'bar'], 5),
 (6, ['atlocation', 'bar', 'bar'], 5),
 (7, ['atlocation', 'bar', 'bar'], 5),
 (8, ['atlocation', 'bar', 'bar'], 5)]

In [40]:
mem_sem

(4, ['atlocation', 'bar', 'bar'], 5)

In [27]:
mem_selected

['atlocation', 'bar', 'bar', 2]